<a href="https://colab.research.google.com/github/FredMiraglia/projeto_analise_dados/blob/main/Nuvem_de_palavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# !pip install notebook wordcloud pillow pandas
#!pip install tqdm

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#################################
from bs4 import BeautifulSoup
import requests
import re
import string



In [56]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

language = 'portuguese'
stop_words = set(stopwords.words(language))
stop_words_customizado = stop_words.union({"a", "ante", "após", "até", "com", "contra", "de", "desde", "em", "entre", "para", "perante", "por", "sem", "sob", "sobre", "trás", "afora", "como", "conforme", "consoante", "durante", "exceto", "mediante", "menos", "salvo", "segundo", "visto",'de',  'a',  'o',  'que',  'e',  'do',  'da',  'em',  'um',  'para',  'é',  'com',  'não',  'uma',  'os',  'no',  'se',  'na',  'por',  'mais',  'as',  'dos',  'como',  'mas',  'foi',  'ao',  'ele',  'das',  'tem',  'à',  'seu',  'sua',  'ou',  'ser',  'quando',  'muito',  'há',  'nos',  'já',  'está',  'eu',  'também',  'só',  'pelo',  'pela',  'até',  'isso',  'ela',  'entre',  'era',  'depois',  'sem',  'mesmo',  'aos',  'ter',  'seus',  'quem',  'nas',  'me',  'esse',  'eles',  'estão',  'você',  'tinha',  'foram',  'essa',  'num',  'nem',  'suas',  'meu',  'às',  'minha',  'têm',  'numa',  'pelos',  'elas',  'havia',  'seja',  'qual',  'será',  'nós',  'tenho',  'lhe',  'deles',  'essas',  'esses',  'pelas',  'este',  'fosse',  'dele',  'tu',  'te',  'vocês',  'vos',  'lhes',  'meus',  'minhas',  'teu',  'tua',  'teus',  'tuas',  'nosso',  'nossa',  'nossos',  'nossas',  'dela',  'delas',  'esta',  'estes',  'estas',  'aquele',  'aquela',  'aqueles',  'aquelas',  'isto',  'aquilo',  'estou',  'está',  'estamos',  'estão',  'estive',  'esteve',  'estivemos',  'estiveram',  'estava',  'estávamos',  'estavam',  'estivera',  'estivéramos',  'esteja',  'estejamos',  'estejam',  'estivesse',  'estivéssemos',  'estivessem',  'estiver',  'estivermos',  'estiverem',  'hei',  'há',  'havemos',  'hão',  'houve',  'houvemos',  'houveram',  'houvera',  'houvéramos',  'haja',  'hajamos',  'hajam',  'houvesse',  'houvéssemos',  'houvessem',  'houver',  'houvermos',  'houverem',  'houverei',  'houverá',  'houveremos',  'houverão',  'houveria',  'houveríamos',  'houveriam',  'sou',  'somos',  'são',  'era',  'éramos',  'eram',  'fui',  'foi',  'fomos',  'foram',  'fora',  'fôramos',  'seja',  'sejamos',  'sejam',  'fosse',  'fôssemos',  'fossem',  'for',  'formos',  'forem',  'serei',  'será',  'seremos',  'serão',  'seria',  'seríamos',  'seriam',  'tenho',  'tem',  'temos',  'tém',  'tinha',  'tínhamos',  'tinham',  'tive',  'teve',  'tivemos',  'tiveram',  'tivera',  'tivéramos',  'tenha',  'tenhamos',  'tenham',  'tivesse',  'tivéssemos',  'tivessem',  'tiver',  'tivermos',  'tiverem',  'terei',  'terá',  'teremos',  'terão',  'teria',  'teríamos',  'teriam'})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
url_base = "https://www.mldb.org"

In [58]:
artista = "artist-6123-o-rappa.html"

In [59]:
req =  requests.get(f'{url_base}/{artista}', headers={"User-Agent": "Mozilla/5.0"})

In [60]:
pag = req.text

In [61]:
sapao = BeautifulSoup(pag, 'html.parser')
sapao

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>MLDb, The Music Lyrics Database - List of songs of O Rappa by albums</title>
<meta content="Nikolay Pelov, NickSoft" name="Author"/>
<meta content="List of songs of O Rappa by albums" name="Description"/>
<meta content="music,song,lyrics,mldb,artist,group,performer,O Rappa,Instinto Coletivo Ao Vivo,Lado B Lado A,O Rappa,O Silêncio Q Precede O Esporro,Rappa-Mundi" name="Keywords"/>
<meta content="index,follow" name="robots"/>
<meta content="7 days" name="revisit-after"/>
<meta content="MLDb, The Music Lyrics Database - List of songs of O Rappa by albums" name="title"/>
<meta content="Global" name="distribution"/>
<meta content="general" name="rating"/>
<link href="style.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/favicon.ico" rel="shortcut icon"/>
<link href="http:

In [62]:
links = sapao.find_all('a', {'href':re.compile(r'song-*')})
len(links)

37

In [63]:
links[4]

<a href="song-165999-a-todas-as-comunidades-do-engenho-novo.html">A Todas As Comunidades Do Engenho Novo</a>

In [64]:
from tqdm import tqdm
letras_radiohead = ''

with open('./lyrics_radiohead.txt','w') as arquivo_letra:
    for links in tqdm(links):
        url_musica = links['href']
        url_completa = f'{url_base}/{url_musica}'
        req = requests.get(url_completa, headers={"User-Agent": "Mozilla/5.0"})
        sapao_musica = BeautifulSoup(req.text, 'html.parser')
        letra = sapao_musica.find('p', {'class':'songtext'})
        arquivo_letra.write(f'{letra.text}\n')
        letras_radiohead += f'{letra.text}\n'




100%|██████████| 37/37 [00:14<00:00,  2.53it/s]


In [65]:
tokken = letras_radiohead.split()
for i in range(len(tokken)):
    tokken[i] = tokken[i].lower().strip()
tokken[:20]





['fica',
 'doido',
 'varrido',
 'quem',
 'quer',
 'se',
 'meter',
 'a',
 'entender',
 'da',
 'mulher',
 'comprei',
 'um',
 'barracão',
 'lá',
 'na',
 'favela',
 'e',
 'botei',
 'no']

In [ ]:
!pip install wordcloud pillow
!pip install STOPWORD

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
palavras_radiohead = ' '.join(tokken)

In [ ]:
palavras_radiohead[100:150]

In [ ]:
wordcloud = WordCloud(background_color='white',
    stopwords = stop_words,
    width = 300,
    height = 200,
    ).generate(palavras_radiohead)

plt.figure(figsize=(10,30))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
